# Imola Programma 2019

## E se Babbo Natale esistesse, e fosse un chatbot?

* **Speaker:** Gabriele Corni [ gabriele_corni@iprel.it ]
* **Azienda:** IPREL Progetti S.r.l
* **Data:** 29 marzo 2019
* **GitHub repository:** https://github.com/horns-g/ImolaProgramma2019.git


### <u>Indice</u>

#### <i>Teoria: applicazioni industriali dei chatbots</i>

[0) Chatbots: chi, come, dove, quando, perche](#intro)

#### <i>Pratica: costruiamo (e proviamo) un chatbot Telegram in 5 minuti</i>

[1) Configurare il file di log](#log)

[2) Definire il comportamento del chatbot](#comportamento)

[3) Creare il chatbot su Telegram](#creazione)

[4) Collegare il chatbot via token](#collegamento)

[5) Effettuare i bindings](#binding)

[6) Proviamolo insieme!](#test)


### Teoria: applicazioni industriali dei chatbots

#### 0) Chatbots: chi, come, dove, quando, perche<a name="intro"></a>

##### Chi

Che cos'è un chatbot, intro sui chatbots

##### Come

steps per la loro creazione: quelli che seguiremo dopo insieme (definire lo scopo, creare le funzioni, associarle, eseguire)

##### Dove

On premises - su cloud (Google DialogFlow, Amazon Lex). in generale su internet (richiedono connessione)

##### Quando

24/7 - un chatbot deve rimanere in esecuzione continua per essere sempre pronto a raggiungere i propri clienti (analogia con server web).

##### Perchè

Maggiore accessibilità, modo "smart" per coinvolgere gli utenti, possibile fare analisi di utilizzo (log), possibile connettere diverse piattaforme anche automaticamente, ... QUI LE APPLICAZIONI INDUSTRIALI


### Pratica: costruiamo (e proviamo) un chatbot Telegram in 5 minuti

#### 1) Configurare il file di log<a name="log"></a>

Abbiamo paragonato un chatbot ad un server web. Ebbene, così come accade per gli accessi ad un sito internet, è parimenti importante loggare gli accessi e le richieste al chatbot.

Questo si rivelerà utile per rilevare malfunzionamenti, monitorare il traffico e soprattutto per analizzare le abitudini di utilizzo degli utenti:

* quali funzioni sono più richieste? 
* quali non vengono quasi mai invocate? 
* cosa potrebbe aumentare il valore percepito dagli utenti sulla base del loro utilizzo?
* ...

In [ ]:
import logging

In [ ]:
logging.basicConfig(
    level=logging.DEBUG,
    format='[ %(asctime)s ] - -  %(levelname)s: %(name)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    filename='chatbot.log',
    filemode='w'
)
console = logging.StreamHandler()
console.setLevel(logging.INFO)
logging.getLogger('').addHandler(console)

#### 2) Definire il comportamento del chatbot<a name="comportamento"></a>

Il comportamento dei chatbot può essere definito tramite l'implementazione di semplici funzioni.

Nel caso di chatbot Telegram implementato in Python, i parametri di ingresso sono generalmente due (entrambi dizionari/oggetti json):

* `bot`: i dettagli del bot che risponde
    * bot_id
    * bot_username
    * bot_first_name
* `update`: i dettagli dell'evento che l'utente ha scatenato interagendo col bot
    * update_id
    * message
        * message_id
        * date
        * chat
            * id
            * type
            * username
            * first_name
            * last_name
        * from
            * id
            * username
            * first_name
            * last_name
            * is_bot
            * language_code
        * text
        * entities
        * caption_entities
        * photo
        * ...
        
Ogni metodo può modificare lo stato interno del bot/del suo backend e/o ritornare un messaggio all'utente relativamente all'esito dell'operazione richiesta.

In quest'ultimo caso, ci si serve sempre dell'oggetto *update*: `update.message.reply_text()`

In [ ]:
def command_1(bot, update):
    """La funzione che incapsula il primo comportamento"""
    update.message.reply_text("command_1")
    logging.info('{} {} {}'.format("command_1", bot, update))


def command_2(bot, update):
    """La funzione che incapsula il secondo comportamento"""
    update.message.reply_text("command_2")
    logging.info('{} {} {}'.format("command_2", bot, update))


def welcome(bot, update):
    """La funzione con cui dare il benvenuto all'utente"""
    update.message.reply_text("welcome")
    logging.info('{} {} {}'.format("welcome", bot, update))


def help_me(bot, update):
    """La funzione con cui spiegare all'utente cosa può fare"""
    update.message.reply_text("help")
    logging.info('{} {} {}'.format("help", bot, update))


def dialog(bot, update):
    """La funzione che risponde al linguaggio naturale inserito dall'utente"""
    update.message.reply_text("dialog")
    logging.info('{} {} {}'.format("dialog", bot, update))


# N.B.: il metodo di errore dettaglia anche gli errori verificatisi
def error(bot, update, error):
    """La funzione che gestisce le eventuali situazioni di errore"""
    update.message.reply_text("error")
    logging.warning('{} {} {} {}'.format("dialog", bot, update, error))

#### 3) Creare il chatbot su Telegram<a name="creazione"></a>

La procedura di creazione va effettuata tramite `@BotFather` via app `Telegram`.

**Solo lo sviluppatore** dovrà preoccuparsi di questa fase.

Si utilizzano i seguenti parametri per la creazione del bot:

##### first_name:

```
imolaprogramma2019
```

##### username: 

```
imolaprogramma2019_bot
```

##### comandi:

```
help - help
command_1 - cmd1
command_2 - cmd2
```

Utilizzare il `token` restituito da `@BotFather` per collegare l'istanza del chatbot ai comportamenti programmati via software.

#### 4) Collegare il chatbot via token<a name="collegamento"></a>

Il token viene copiato in chiaro nel sorgente per motivi di tempo e per fini dimostrativi.

Sarebbe più opportuno salvare il token come variabile d'ambiente e caricarne il valore trasparentemente.

In caso di applicazioni reali è caldamente consigliata l'adozione di adeguate misure di sicurezza.

In [ ]:
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

In [ ]:
TELEGRAM_BOT_TOKEN = '722180203:AAEBCSejdIlWJbUNv0CLsaa3M6FY0Q0SV7U'

In [ ]:
# Istanziare l'EventHandler del chatbot creato passandogli il codice identificativo.
updater = Updater(TELEGRAM_BOT_TOKEN)

In [ ]:
# Ottenere il dispatcher di eventi.
dp = updater.dispatcher

#### 5) Effettuare i bindings<a name="binding"></a>

L'ultima cosa che resta da fare è associare ogni comando del chatbot ad una delle funzioni precedentemente definite.

In [ ]:
# Associare al dispatcher i comportamenti del chatbot:
# # Comandi (risposte a necessità utente)
dp.add_handler(CommandHandler("start", welcome))
dp.add_handler(CommandHandler("help", help_me))

dp.add_handler(CommandHandler("command_1", command_1))
dp.add_handler(CommandHandler("command_2", command_2))

# # Comprensione del linguaggio naturale (dialogo con utente)
dp.add_handler(MessageHandler(Filters.text, dialog))

# # Errori (notificare errori nell'utilizzo o occorsi)
dp.add_error_handler(error)

#### 6) Proviamolo insieme!<a name="test"></a>

**Lato Python:** avviare e mantenere attivo il chatbot finchè il processo non viene interrotto

In [ ]:
logging.info('Bot avviato')

updater.start_polling()
updater.idle()

logging.info('Bot spento')

**Lato Telegram:** aprire l'app, cercare dalla lente di ingrandimento l'utente `imolaprogramma2019` ed avviare una sessione di utilizzo premendo il pulsante `START` in basso.

* Il bot accoglierà ogni nuovo utente con il messaggio di `welcome()`
* I comandi configurati saranno accessibili dall'apposito tasto `/` di fianco alla chat
* L'interazione in linguaggio naturale verrà gestita dalla logica di `dialog()`
* Eventuali errori verranno gestiti e notificati dal metodo di `error()`